In [1]:
# Import the neccesary libraries needed
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
from numpy import insert
import os
import time

# keras tuner for hyperparameter tuning
import keras_tuner as kt

# tensorflow Libraries
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Import the CSV Training and Testing Data Sets

In [2]:
# Import the csv's (IMPORTANT: You need at least 16 Gb of RAM to proceed)

# Original Dataset
#train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\PortTraining_1404_002_FINAL_startingAt24.csv')
#test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\StbdTesting_1404_002_FINAL.csv')

# Deeper More Variate Dataset
#train38 = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port.csv')
#test38 = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd.csv')

# Noise Reduction RAW Dataset
#train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_port.csv')
#test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_stbd.csv')

# Noise Reduction Dataset
train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port_FinalCleaned_RollReady.csv')
test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd_FinalCleaned_RollReady.csv')

#train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\repackingTestFiles\\0001_1404.038_port_new.csv')
#test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\repackingTestFiles\\0001_1404.038_stbd_new.csv')

In [ ]:
train

In [ ]:
test

Split up the Features and Labels from both the Training and Testing Datasets

In [3]:
# Split up the features and labels from both the training and testing datasets
#x_train=train.iloc[:,4:24]

#x_train=train.iloc[:,3:24] # sample time delay through Q10

x_train=train.iloc[:,3:25] # sample time delay through roll
#x_train=train38.iloc[:,3:24]

x_train['Amplitude']=train.iloc[:,29]
#x_train['Amplitude']=train38.iloc[:,29]


#x_test=test.iloc[:,4:24]
#x_test=test.iloc[:,3:24]    # sample time delay through Q10

x_test=test.iloc[:,3:25]    # sample time delay through roll
x_test['Roll'] = x_test['Roll']


x_test['Amplitude']=test.iloc[:,29]
#x_test['Amplitude']=test38.iloc[:,29]

y_train=train.iloc[:,26]
#y_train=train38.iloc[:,26]

y_test=test.iloc[:,26]
#y_test=test38.iloc[:,26]

In [ ]:
y_test

In [5]:
x_train

,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,...,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,Amplitude
0,0.031901,11.46875,50.12500,-49.75000,-41.718750,72.93750,7.437500,-63.062500,35.437500,27.281250,...,-52.562500,-40.156250,60.656250,6.218750,-41.343750,40.375000,19.562500,-68.031250,1.043701,21
1,0.031916,3.62500,13.34375,-16.03125,-16.656250,24.68750,10.093750,-19.875000,4.156250,3.312500,...,-10.031250,1.406250,4.968750,-6.281250,8.187500,7.375000,-13.531250,-0.375000,1.043701,24
2,0.031947,-15.84375,37.25000,-14.12500,-40.781250,43.09375,28.718750,-56.375000,9.750000,43.218750,...,-35.750000,-55.312500,65.218750,27.218750,-67.156250,24.968750,45.875000,-72.062500,1.043701,43
3,0.031962,-3.31250,121.34375,-96.25000,-112.593750,156.46875,47.968750,-151.093750,55.812500,71.281250,...,-119.437500,-94.500000,137.750000,8.562500,-91.656250,98.187500,24.437500,-156.968750,1.043701,14
4,0.031977,44.46875,86.15625,-118.75000,-51.562500,134.46875,-18.375000,-85.968750,91.781250,-10.500000,...,-126.281250,-4.875000,90.031250,-65.625000,1.125000,109.062500,-77.750000,-102.375000,1.043701,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635094,0.099403,-26.40625,6.56250,-3.90625,18.796875,5.90625,9.125000,7.593750,6.531250,14.375000,...,-9.828125,-16.390625,-19.718750,2.109375,-1.765625,12.890625,6.500000,7.578125,0.758057,14
3635095,0.099419,-40.06250,16.34375,-11.12500,38.437500,20.62500,33.406250,41.968750,4.953125,28.296875,...,-38.250000,-13.375000,-35.125000,19.359375,-3.390625,35.953125,28.390625,24.609375,0.758057,18
3635096,0.099434,-16.90625,18.75000,-4.96875,29.453125,29.06250,31.421875,44.828125,-13.218750,6.796875,...,-33.703125,9.156250,-18.171875,20.140625,-6.890625,36.171875,40.296875,27.734375,0.758057,19
3635097,0.099450,40.43750,28.31250,41.90625,-6.828125,31.34375,-35.531250,-15.078125,-51.843750,-57.734375,...,7.484375,54.796875,41.265625,18.406250,29.671875,-9.234375,20.515625,-36.531250,0.758057,22


In [ ]:
x_test

Scale the Features from -1 to 1 so that Training is Easier for the Model

In [6]:
# Scale the Features and Labels from [-1,1]

s1=MinMaxScaler(feature_range=(0,1))
#s1=MinMaxScaler(feature_range=(0,1))
x_train_scale=s1.fit_transform(x_train)

#s2=MinMaxScaler(feature_range=(0,1))
#s2=MinMaxScaler(feature_range=(0,1))
x_test_scale=s1.transform(x_test)
x_test_scale=x_test_scale
x_test_scale[:, 21] = -x_test_scale[:, 21]
# Scale DOA's from (0 to 1)
#s3=MinMaxScaler(feature_range=(0,1))
#y_train_scale = s3.fit_transform(train[['DOA']])
y_train_scale= y_train.to_numpy()

#s4=MinMaxScaler(feature_range=(0,1))
#y_test_scale = s4.fit_transform(test[['DOA']])
y_test_scale= y_test.to_numpy()

print(len(x_train_scale))
print(len(y_train_scale))

3635099
3635099


In [9]:
x_train_scale

array([[0.03403601, 0.55510042, 0.49083105, ..., 0.44995814, 0.59866221,
        0.19230769],
       [0.0342556 , 0.55140201, 0.47341079, ..., 0.48352091, 0.59866221,
        0.22115385],
       [0.03469477, 0.54222229, 0.48473322, ..., 0.44795833, 0.59866221,
        0.40384615],
       ...,
       [0.99956083, 0.54172131, 0.47597129, ..., 0.49746535, 0.42474916,
        0.17307692],
       [0.99978041, 0.56875949, 0.48050026, ..., 0.4655846 , 0.42474916,
        0.20192308],
       [1.        , 0.57017402, 0.4762377 , ..., 0.44933805, 0.42474916,
        0.21153846]])

In [7]:
x_train_scale[:,21]

array([0.59866221, 0.59866221, 0.59866221, ..., 0.42474916, 0.42474916,
       0.42474916])

In [8]:
x_test_scale[:,21]

array([-0.59866221, -0.59866221, -0.59866221, ..., -0.42474916,
       -0.42474916, -0.42474916])

In [ ]:
y_train_scale

In [ ]:
Test_DOA_df= pd.DataFrame(y_test_scale, columns=['TEST DOAs'])
Test_DOA_df


In [ ]:
Test_DOA_df.describe()

In [10]:
df= pd.DataFrame(x_train_scale, columns= ['TWTT','I1', 'Q1', 'I2', 'Q2', 'I3', 'Q3', 'I4', 'Q4', 'I5', 'Q5', 'I6', 'Q6', 'I7', 'Q7', 'I8', 'Q8', 'I9', 'Q9', 'I10', 'Q10','Roll', 'Amplitude'])
df


,TWTT,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,...,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,Amplitude
0,0.034036,0.555100,0.490831,0.478568,0.419284,0.604678,0.503978,0.500413,0.625890,0.401728,...,0.457680,0.513242,0.447324,0.481483,0.493715,0.458495,0.565826,0.449958,0.598662,0.192308
1,0.034256,0.551402,0.473411,0.494694,0.431328,0.584435,0.505259,0.520081,0.613071,0.391703,...,0.477378,0.531799,0.423836,0.474472,0.518744,0.445072,0.551515,0.483521,0.598662,0.221154
2,0.034695,0.542222,0.484733,0.495606,0.419734,0.592157,0.514239,0.503458,0.615363,0.408393,...,0.465466,0.506474,0.449248,0.493261,0.480671,0.452228,0.577205,0.447958,0.598662,0.403846
3,0.034914,0.548131,0.524562,0.456328,0.385222,0.639724,0.523520,0.460322,0.634240,0.420130,...,0.426706,0.488977,0.479840,0.482797,0.468291,0.482013,0.567934,0.405838,0.598662,0.125000
4,0.035134,0.570660,0.507896,0.445567,0.414553,0.630493,0.491532,0.489981,0.648981,0.385926,...,0.423537,0.528995,0.459713,0.441188,0.515175,0.486436,0.523744,0.432921,0.598662,0.201923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635094,0.999122,0.537242,0.470199,0.500493,0.448367,0.576555,0.504791,0.532590,0.614044,0.396330,...,0.477472,0.523853,0.413423,0.479178,0.513715,0.447315,0.560177,0.487466,0.424749,0.125000
3635095,0.999341,0.530803,0.474832,0.497041,0.457806,0.582730,0.516499,0.548245,0.613397,0.402153,...,0.464308,0.525200,0.406925,0.488853,0.512894,0.456697,0.569644,0.495915,0.424749,0.163462
3635096,0.999561,0.541721,0.475971,0.499985,0.453488,0.586270,0.515542,0.549547,0.605950,0.393161,...,0.466414,0.535260,0.414076,0.489291,0.511125,0.456786,0.574793,0.497465,0.424749,0.173077
3635097,0.999780,0.568759,0.480500,0.522404,0.436052,0.587227,0.483260,0.522265,0.590121,0.366171,...,0.485490,0.555638,0.439145,0.488318,0.529601,0.438315,0.566238,0.465585,0.424749,0.201923


In [ ]:
df.describe()

Shift the DOA's to the Left by one so that the Current I and Q data Align with the current DOA while using TimeSeriesGenerator to Window the Data

In [ ]:
# Shift the DOA to the left by one to use the current DOA label with the current I and Q data.
# Normally the TimeseriesGenerator function uses past values to predict the future but we would like current data to help predict the current samples' DOA
print(len(y_train_scale))
print(len(y_test_scale))
y_train_scale=insert(y_train_scale, 0, 0)
y_train_scale=np.delete(y_train_scale, -1)
y_test_scale=insert(y_test_scale, 0,0)
y_test_scale=np.delete(y_test_scale, -1)
print(y_train_scale)
print(len(y_train_scale))
print(y_test_scale)
print(len(y_test_scale))

Create the Windows Neccesary for the LSTM model using the TimeSeriesGenerator Function

In [ ]:
# Create the windows neccesary for the LSTM model within tensorflow keras
# Below are the arguments of the timeseries_dataset_from_array function
# The function takes a numpy array and makes a timeseries out of it
NumSampsPerPing= 4301-24+1
data=x_train_scale # the data to make the windows
targets=y_train_scale # time steps in the data (don't need)
sequence_length=12 # window length
sequence_stride=1# period between successive output sequences
sampling_rate=1 # period between successive individual timesteps     within sequences
batch_size=8 # number of time series samples in each batch
shuffle=False #shuffle the data before making the windows
seed=None # is related to shuffle
start_index=None # is related to shuffle
end_index=None # is related to shuffle
n_features=21

test_data=x_test_scale
test_targets=y_test_scale

dataRows = data.shape[0]
dataRows

In [ ]:
#validationStartRow = dataRows - NumSampsPerPing*281
validationStartRow = dataRows - 2400*1000
#validationStartRow = dataRows - 8*1000

print(validationStartRow, ',',dataRows)

train_data = data[:validationStartRow-1]
#train_data = data
#   train_data.shape[0]
train_targets = targets[:validationStartRow-1]
#train_targets = targets
#   train_targets.shape[0]
val_data = data[validationStartRow:]
#   val_data.shape[0], dataRows-validationStartRow
y_val_scale = y_train_scale[validationStartRow:]

val_targets = targets[validationStartRow:]

In [ ]:
train_data.shape , train_targets.shape

In [ ]:
val_data.shape , val_targets.shape

In [ ]:


#inputs=TimeseriesGenerator(data, targets,sequence_length, batch_size)
train_inputs=TimeseriesGenerator(data=train_data,targets=train_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size, shuffle=True)
trainentire_inputs = TimeseriesGenerator(data=data,targets=targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256)

val_inputs = TimeseriesGenerator(data=val_data,targets=val_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256, shuffle=False)

#test_inputs=TimeseriesGenerator(test_data[0:40000], -1*test_targets[0:40000],sequence_length, batch_size)
test_inputs=TimeseriesGenerator(data=test_data,targets=test_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256)

#inputs=tf.keras.preprocessing.timeseries_dataset_from_array(data, targets, sequence_length, sequence_stride, sampling_rate, batch_size, shuffle)
#print(type(inputs))

#print('done')

In [ ]:
# abstraction test imports
abstraction_test_port = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port.csv')
abstraction_test_stbd = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd.csv')
# Split up the features and labels from both the training and testing datasets
#x_train=train.iloc[:,4:24]

Xatest_port=abstraction_test_port.iloc[:,3:24]
#x_train['TWTT']=train.iloc[:,28]  I feel as though the TWTT has NO impact on the DOA so we should NOT include it in training
Xatest_port['Amplitude']=abstraction_test_port.iloc[:,29]


#x_test=test.iloc[:,4:24]
Xatest_stbd=abstraction_test_stbd.iloc[:,3:24]
#x_test['TWTT']=test.iloc[:,28] I feel as though the TWTT has NO impact on the DOA so we should NOT include it in training
Xatest_stbd['Amplitude']=abstraction_test_stbd.iloc[:,29]

Yatest_port=abstraction_test_port.iloc[:,26]

Yatest_stbd=abstraction_test_stbd.iloc[:,26]

s10=MinMaxScaler(feature_range=(0,1))
#s1=MinMaxScaler(feature_range=(0,1))
Xatest_port_scale=s10.fit_transform(Xatest_port)

s20=MinMaxScaler(feature_range=(0,1))
#s2=MinMaxScaler(feature_range=(0,1))
Xatest_stbd_scale=s20.fit_transform(Xatest_stbd)
#x_test_scale=x_test_scale

# Scale DOA's from (0 to 1)
s30=MinMaxScaler(feature_range=(0,1))
#y_train_scale = s3.fit_transform(train[['DOA']])
Yatest_port_scale= Yatest_port.to_numpy()

s40=MinMaxScaler(feature_range=(0,1))
#y_test_scale = s4.fit_transform(test[['DOA']])
Yatest_stbd_scale= Yatest_stbd.to_numpy()

print(len(Xatest_port_scale))
print(len(Xatest_stbd_scale))

print(len(Yatest_port_scale))
print(len(Yatest_stbd_scale))
Yatest_port_scale=insert(Yatest_port_scale, 0, 0)
Yatest_port_scale=np.delete(Yatest_port_scale, -1)
Yatest_stbd_scale=insert(Yatest_stbd_scale, 0,0)
Yatest_stbd_scale=np.delete(Yatest_stbd_scale, -1)
print(Yatest_port_scale)
print(len(Yatest_port_scale))
print(Yatest_stbd_scale)
print(len(Yatest_stbd_scale))


abstraction_test_port_inputs=TimeseriesGenerator(data=Xatest_port_scale,targets=Yatest_port_scale,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size)
abstraction_test_stbd_inputs=TimeseriesGenerator(data=Xatest_stbd_scale,targets=Yatest_stbd_scale,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size)




In [ ]:
# print the first couple of samples to see if everything is lining up
for i in range(100):
	x, y = train_inputs[i]
	print('%s => %s' % (x, y))

In [ ]:
# print the first couple of samples to see if everything everything is lining up
for i in range(100):
	x, y = test_inputs[i]
	print('%s => %s' % (x, y))

Define the Model's Structure

In [ ]:
# clear the model!
model = 0

In [ ]:
# With all the preproccessing done, it is time to define the model
# Define the LSTM Model
model= Sequential(name='MLDOASequential')
#model.add(LSTM(units=48, name='LSTM1', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=True))
model.add(LSTM(units=48, name='LSTM2', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=False))
#model.add(LSTM(units=44, name='LSTM1', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=False))

#model.add(LSTM(units=64, name='LSTM1', activation='tanh', batch_input_shape=(batch_size, sequence_length, n_features),input_shape=(sequence_length,n_features), return_sequences=True, stateful=False, bias_initializer='ones'))
#model.add(LSTM(units=64, name='LSTM1', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=True, stateful=False, bias_initializer='Ones'))
#model.add(Dense(units=))
#model.add(Dense(units=300, name='Dense1', activation = 'relu'))
#model.add(Dropout(0.2, name='Dropout1'))

#model.add(LSTM(units=416, name='LSTM2', activation='tanh', return_sequences=True, stateful=False, bias_initializer='ones'))
#model.add(Dense(units=500, name='Dense2'))
#model.add(Dense(units=100, activation='relu'))
#model.add(Dropout(0.1, name='Dropout2'))

#model.add(LSTM(units=312, name='LSTM3', activation='tanh', return_sequences=False, stateful=False, bias_initializer='Ones'))
#model.add(Dense(units=300, name='Dense3', activation='relu'))

#model.add(Dropout(0.3, name='Dropout1'))

#model.add(Dense(units=10, name='Dense4', activation='relu', input_shape=(sequence_length,n_features)))
model.add(Dense(units=96, name='Dense1', activation='relu'))
#model.add(Dropout(0.2, name='Dropout2'))
model.add(Dense(units=96, name='Dense2', activation='relu'))
#model.add(Dropout(0.3, name='Dropout3'))
model.add(Dense(units=96, name='Dense3', activation='relu'))
#model.add(Dropout(0.2, name='Dropout3'))
#model.add(Dense(units=30, name='Dense4', activation='relu'))
#model.add(Dropout(0.2, name='Dropout4'))
#model.add(Dense(units=30, name='Dense5', activation='relu'))
#model.add(Dropout(0.2, name='Dropout5'))
#model.add(Dense(units=10, name='Dense9', activation='relu'))
#model.add(Dropout(0.2, name='Dropout6'))
#model.add(LSTM(units=84, name='LSTM3', activation='tanh', return_sequences=True))
#model.add(Dense(units=100))
#model.add(Dense(units=100, activation = 'linear'))
#model.add(Dropout(0.2))
#model.add(LSTM(units=320, activation='tanh', return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(units=168, name='LSTM4', activation='tanh'))
#model.add(Dense(units=100))
#model.add(Dense(units=100, activation = 'linear'))
#model.add(Dropout(0.2))
#model.add(LSTM(units=480))
#model.add(Dropout(0.2))


model.add(Dense(units=1, name='DenseOutput', activation='linear')) #one output (namely: DOA)



In [ ]:
model.summary()

Train the Model using the Specified Hyperparmeters

In [ ]:
# epoch learning rate scheduler for 10 epochs.
def EpochScheduler10_2(epoch, lr):
    if epoch == 0:
        return 0.1
    if epoch == 1:
        return 0.1
    elif epoch == 2:
        return 0.1
    elif epoch == 3:
        return 0.05
    elif epoch == 4:
        return 0.05
    elif epoch == 5:
        return 0.05
    elif epoch == 6:
        return 0.025
    elif epoch == 7:
        return 0.025
    elif epoch == 8:
        return 0.025
    elif epoch == 9:
        return 0.01
    elif epoch == 10:
        return 0.01
    elif epoch == 11:
        return 0.01
    else:
        return 0.01


In [ ]:
# epoch learning rate scheduler for 10 epochs.
def EpochScheduler10(epoch, lr):
    if epoch == 0:
        return 0.01
    if epoch == 1:
        return 0.01
    elif epoch == 2:
        return 0.01
    elif epoch == 3:
        return 0.005
    elif epoch == 4:
        return 0.005
    elif epoch == 5:
        return 0.005
    elif epoch == 6:
        return 0.0025
    elif epoch == 7:
        return 0.0025
    elif epoch == 8:
        return 0.0025
    elif epoch == 9:
        return 0.001
    elif epoch == 10:
        return 0.001
    elif epoch == 11:
        return 0.001
    else:
        return 0.001


In [ ]:
def scheduler(epoch,lr):
    if epoch<=1:
        return lr
    elif epoch>=2 | epoch<=4:
        #return lr/1.65
        return lr/1.55
    else:
        #return lr * tf.math.exp(-0.5)
        return lr * tf.math.exp(-0.4)

In [ ]:
#Now it is time to train the model
#opt=tf.keras.optimizers.Adam(learning_rate=0.00007)#,decay=1e-5)
opt=tf.keras.optimizers.Adam(learning_rate=2)#,decay=1e-5)
#opt = tf.keras.optimizers.Adam()
mMSE = tf.keras.metrics.MeanSquaredError()     # metric for Mean Squared Error
mRMSE = tf.keras.metrics.RootMeanSquaredError()
huberLoss = tf.keras.losses.Huber(delta=10)
#model.compile(loss='mse', optimizer=opt, metrics=[mMSE, mRMSE])
model.compile(loss=huberLoss, optimizer=opt, metrics=[mMSE, mRMSE])

# Allow for early stopping so that the model does not overfit the training dataset
#es= EarlyStopping(monitor='val_loss', mode='min',verbose=1,patience=10)
es= EarlyStopping(monitor='loss', mode='min',verbose=1,patience=10)
# change learning rates per epoch
lrCallback = tf.keras.callbacks.LearningRateScheduler(EpochScheduler10_2)
sc = tf.keras.callbacks.LearningRateScheduler(scheduler)
# Model Checkpoint to save good runs
#cp = ModelCheckpoint('model/', save_best_only=True)
cp = ModelCheckpoint('model/', monitor='loss',save_best_only=True)

t0=time.time()
#history= model.fit(inputs,steps_per_epoch=4000,epochs=200,verbose=1, callbacks=[es])
#history= model.fit(train_inputs, steps_per_epoch=4000, validation_data=val_inputs, epochs=100, callbacks=[lrCallback, cp])
#history= model.fit(train_inputs, epochs=1, batch_size=None,callbacks=[cp, es])
#history= model.fit(train_inputs, epochs=75, steps_per_epoch=1782, validation_steps=10720,validation_data=val_inputs, batch_size=None,callbacks=[cp])
#history= model.fit(train_inputs, epochs=15,validation_data=val_inputs, batch_size=None,callbacks=[cp, sc])

#history = model.fit(train_inputs, epochs = 60, steps_per_epoch = 100, validation_data=val_inputs, batch_size=None,callbacks=[cp, es])
history = model.fit(train_inputs, epochs = 15, steps_per_epoch = 1500, batch_size=None,callbacks=[cp, es])

t1=time.time()
print("The total run time to train was %.2f seconds"%(t1-t0))

# plot the loss function


In [ ]:
plt.figure()
plt.semilogy(history.history['loss'])
plt.semilogy(history.history['val_loss'])
#plt.semilogy(history.history['lr'])
plt.xlabel('epoch'); plt.ylabel('loss')


In [ ]:
tf.keras.models.save_model(model, filepath="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\SavedModels\\MB\\Mar10_2022\\model_row38.h5")

In [ ]:
model = keras.models.load_model('model/')
#model = keras.models.load_model('model_3_13_22_epoch4/model/')
#model = keras.models.load_model(filepath="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\SavedModels\\MB\\Mar10_2022\\model_row12.h5")

In [ ]:
#trainpredicted=model.predict(train_inputs, verbose=1)
#valpredicted=model.predict(val_inputs, verbose=1)
trainentirepredicted=model.predict(trainentire_inputs, verbose=1)
predicted=model.predict(test_inputs, verbose=1)
#trainpredicted.shape, valpredicted.shape, predicted.shape, trainentirepredicted.shape

In [ ]:
predicted.shape, trainentirepredicted.shape

In [ ]:
port_abstraction_test_predicted=model.predict(abstraction_test_port_inputs, verbose=1)

In [ ]:
Yatest_port_scale
#val_targets
plt.figure(facecolor='white')
plt.plot(Yatest_port_scale[0:4278*3])
plt.plot(port_abstraction_test_predicted[0:4278*3])
plt.title("Port Abstraction Test (Actual vs. Predicted)")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()

In [ ]:
trainpredicted.shape, valpredicted.shape,trainpredicted.shape[0]+valpredicted.shape[0], predicted.shape, trainentirepredicted.shape

In [ ]:
predicted=model.predict(test_inputs, verbose=1)

In [ ]:
#val_targets
plt.figure(facecolor='white')
plt.plot(val_targets[0:2278*3])
plt.plot(valpredicted[0:2278*3])
plt.title("Validation (Actual vs. Predicted)")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()

In [ ]:
plt.figure(facecolor='white')
#plt.plot(y_train[0:2278*3])
plt.plot(y_train[0:80000])
#plt.plot(trainentirepredicted[0:2278*3])
plt.plot(trainentirepredicted[0:80000])
plt.title("Training (Actual vs. Predicted)")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()

In [ ]:
zerosArray= np.arange(sequence_length)*0
predictedShifted=np.append(zerosArray, predicted)
plt.figure(facecolor='white')
#plt.plot(y_test[0:4278*4])
plt.plot(y_test[0:80000])
#plt.plot(predictedShifted[0:4278*4])
plt.plot(predictedShifted[0:80000])
plt.title("Testing (Actual vs. Predicted)")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()



Using the Model, Predict the DOA's on the Testing Dataset

In [ ]:
#use the model to predict on the testing dataset
#for right now test_inputs has only the 40,001 samples
predicted=model.predict(test_inputs, verbose=1)

In [ ]:
predicted.shape

Put the predicted DOA's back into a csv file format

In [ ]:
#Ping |  Num Samp Num | PORT or STBD (0 or 1) |  TWTT |  Predicted DOA   <=======Output columns.
OutputCSVdf=test.iloc[:,[0,1,2,3]]
#OutputCSVdf= pd.DataFrame(test.iloc[:,[0,1,2,3]], columns=['PingNumber','SampNumber','PortStbd', 'SampleTime'])
zerosArray= np.arange(sequence_length)*0
predictedShifted=np.append(zerosArray, predicted)
Predicteddf= pd.DataFrame(predictedShifted,columns=['PredictedDOA'])
OutputCSVdf=OutputCSVdf.join(Predicteddf)

In [ ]:
OutputCSVdf.to_csv(path_or_buf="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB37.csv")

In [ ]:
PortOutputCSVdf=train.iloc[:,[0,1,2,3]]
#OutputCSVdf= pd.DataFrame(test.iloc[:,[0,1,2,3]], columns=['PingNumber','SampNumber','PortStbd', 'SampleTime'])
zerosArray= np.arange(sequence_length)*0
portPredictedShifted=np.append(zerosArray, trainentirepredicted)
portPredicteddf= pd.DataFrame(portPredictedShifted,columns=['PredictedDOA'])
PortOutputCSVdf=PortOutputCSVdf.join(portPredicteddf)
PortOutputCSVdf.to_csv(path_or_buf="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTrainMB37.csv")

In [ ]:
StbdOriginalOutputCSVdf = test.iloc[:,[0,1,2,3,26]]
StbdOriginalOutputCSVdf.to_csv(path_or_buf="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\Stbd_038_forEric.csv")
PortOriginalOutputCSVdf = train.iloc[:,[0,1,2,3,26]]
PortOriginalOutputCSVdf.to_csv(path_or_buf="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\Port_038_forEric.csv")

In [ ]:
test.iloc[:,[0,1,2,3]]

In [ ]:
zerosArray= np.arange(sequence_length)*0
zerosArray

In [ ]:
predictedShifted=np.append(zerosArray, predicted)
predictedShifted

In [ ]:
OutputCSVdf

In [ ]:
print(predicted[0:21])

In [ ]:
print(y_test[10:21])

In [ ]:
type(predicted)
predicted.shape

In [ ]:
predicted_reshape= predicted.reshape(len(predicted),1)

In [ ]:
plt.figure(facecolor='white')
plt.plot(predicted[0:40000], 'orange')
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Predicted'])
plt.show()

Compare the Actual Testing DOA's to the Predicted DOA's

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)


plt.figure(facecolor='white')
plt.plot(y_test[0:10], '.')
plt.plot(predicted[0:10], '.')
plt.title("Actual")
plt.legend(['Actual','Predicted'])
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.show()

plt.figure(facecolor='white')
plt.plot(y_test[0:40000])
plt.plot(predicted_reshape[0:40000])
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()


plt.figure(facecolor='white')
plt.plot(y_test[0:40000])
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual'])
plt.show()


plt.figure(facecolor='white')
plt.plot(predicted_reshape[0:40000], 'orange')
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Predicted'])
plt.show()


In [ ]:
print(y_test[0:50000])


In [ ]:
df2= pd.DataFrame(predicted, columns= ['DOA'])
df2